In [1]:
import os
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torchvision import transforms
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint
from libs.code import *
from pytorch_lightning.loggers import TensorBoardLogger
import torch
import pytorch_lightning as pl
import faiss

from libs.Dataset import *
from libs.util import *
from libs.SiameseNetwork import TripletNetworkTask
# non necessari !
# from libs.code import *
# from libs.VAE import *

Imposto i seed e le variabili globali:

In [2]:
random.seed(1996)
np.random.seed(1996)
pl.seed_everything(1996)

# Ensure that all operations are deterministic on GPU (if used) for reproducibility
torch.backends.cudnn.determinstic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print("Device:", device)

Global seed set to 1996


Device: cpu


In [3]:
PATH_DST = 'dataset/all_labels.csv'
PATH_GDRIVE = ''
# TODO: se setto > 0 mi da 
# [W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
# e non mi permette di effettuare il training. tuttavia resta troppo lento. come procedo?
NUM_WORKERS = 12
BATCH_SIZE = 128

GPUS = 0
PRETRAINED_MODEL_PATH =  'models/squeezeNet_pretrained.pth'
num_class = 3

# valori pretrained
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225] 

In [5]:
# transf = transforms.Compose([transforms.Resize((224,224)), transforms.ToTensor(), transforms.Normalize(mean=mean, std=std)])
from torch.nn import ModuleList

transf_train = transforms.Compose([
        transforms.Resize(230), # taglio solo una piccola parte col randomCrop in modo tale da prendere sempre il secchio
        transforms.RandomCrop(224),
        transforms.RandomApply(ModuleList([
            transforms.ColorJitter(brightness=.3, hue=.2),
        ]), p=0.3),
        transforms.RandomGrayscale(p=0.2),
        transforms.RandomHorizontalFlip(p=0.3),
        transforms.RandomPerspective(distortion_scale=0.3, p=0.2),
        transforms.RandomEqualize(p=0.2),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std)
    ])


transf_test = transforms.Compose([
        transforms.Resize(256), 
        transforms.CenterCrop(224), 
        transforms.AutoAugment(transforms.AutoAugmentPolicy.SVHN),
        transforms.RandomInvert(p=0.3),
        transforms.RandomHorizontalFlip(p=0.2),
        transforms.RandomGrayscale(p=0.2),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std)
    ])

#### Carico il dataset singolo

In [6]:
from sklearn.model_selection import train_test_split

df = pd.read_csv(PATH_DST)

df_train, df_test = train_test_split(df, test_size=0.20, random_state=0)

print("df_train: {} , df_test: {}, is splitted correctly: {}".format(len(df_train), len(df_test), (len(df) == (len(df_test)+len(df_train)) )))

df_train.to_csv("dataset/df_training.csv")
df_test.to_csv("dataset/df_test.csv")

df_train: 10560 , df_test: 2640, is splitted correctly: True


In [6]:
dst_train = TrashbinDataset(csv=PATH_DST, transform=transf_train)
dst_test = TrashbinDataset(csv=PATH_DST, transform=transf_test)


dst_train_loader = DataLoader(dst_train, num_workers=NUM_WORKERS, batch_size=BATCH_SIZE, shuffle=True)
dst_test_loader = DataLoader(dst_test, num_workers=NUM_WORKERS, batch_size=BATCH_SIZE, shuffle=False)

##### Estrazione delle rappresentazioni e prime predizioni con nn
Estraggo le rappresentazioni rgb dai loader: <b>TODO:</b> non ho ancora eseguito con la data augment

In [5]:
dst_train_rep_rgb, dst_train_labels = extract_rgb_representations(loader=dst_train_loader)
dst_test_rep_rgb, dst_test_labels = extract_rgb_representations(loader=dst_test_loader)

100%|██████████| 83/83 [01:25<00:00,  1.04s/it]


Rappresentazioni di training:

In [6]:
dst_train_rep_rgb.shape

(10560, 150528)

Ottengo le predizioni sul test-set usando `predict_nn`:

In [7]:
pred_test_label_rgb = predict_nn(dst_train_rep_rgb, dst_test_rep_rgb, dst_train_labels)
print(f"Sample di label: {pred_test_label_rgb}")

Sample di label: [2 0 0 ... 0 2 2]


Valuto le performance della baseline

In [8]:
classification_error = evaluate_classification(pred_test_label_rgb, dst_test_labels)
print(f"Classification error: {classification_error:0.2f}")

Classification error: 5.29


#### Modello

<s>Importo per effettuare il training della triplenet il miglior modello trovato nella precedente relazione: `SqueezeNet v1.0`. Importo dunque i pesi già trovati dopo il training di 100 epoche .... <b>TODO migliora la descrizione</b> ... importo i pesi.. faccio le opportune modifiche ...</s>

In [4]:
# TODO: Vorrei usare il modello già allenato precedentemente. Ma come?
# scarico il modello da pytorch
squeezeNet_1_0 = torch.hub.load('pytorch/vision:v0.10.0', 'squeezenet1_0', pretrained=True)
# applico le opportune modifiche
squeezeNet_1_0.classifier[1] = nn.Conv2d(512, num_class, kernel_size=(1,1), stride=(1,1))
# # # carico i pesi salvati

squeezeNet_1_0.load_state_dict(torch.load(PRETRAINED_MODEL_PATH))

# testo così
squeezeNet_1_0.classifier = nn.Sequential(
    # nn.Dropout(p=0.5, inplace=False),
    # nn.Conv2d(512, num_class, kernel_size=(1, 1), stride=(1, 1)),
    # nn.Conv2d(512, 1000, kernel_size=(1, 1), stride=(1, 1))
    nn.ReLU(inplace=True),
    nn.AdaptiveAvgPool2d(output_size=(1, 1)),
    nn.Identity()
  )

squeezeNet_1_0(torch.zeros(1, 3, 224,224)).shape

Using cache found in /Users/danilo/.cache/torch/hub/pytorch_vision_v0.10.0


torch.Size([1, 512])

In [10]:
# uso il modello, allenato nel precedente progetto, per estrarre le rappresentazioni dal training e dal test set
dst_train_rep, dst_train_labels = extract_rep_squeezeNet(squeezeNet_1_0, dst_train_loader)
dst_test_rep, dst_test_labels = extract_rep_squeezeNet(squeezeNet_1_0, dst_test_loader)

100%|██████████| 83/83 [03:09<00:00,  2.29s/it]


Valuto le performance del sistema:

In [11]:
# valuto le performance del sistema con rappresentazioni non ancora ottimizzate
pred_test_label = predict_nn(dst_train_rep, dst_test_rep, dst_train_labels)
classification_error = evaluate_classification(pred_test_label, dst_test_labels)
print(f"Classification error: {classification_error}")

Classification error: 0.0


#### Carico il dataset in triplette:

In [5]:
transf = transforms.Compose([transforms.Resize((224,224)), transforms.ToTensor(), transforms.Normalize(mean=mean, std=std)])

# non funzionano !!! non allenano il modello!

# transf_train = transforms.Compose([
#         transforms.Resize(230), # taglio solo una piccola parte col randomCrop in modo tale da prendere sempre il secchio
#         transforms.RandomCrop(224),
#         transforms.RandomApply(ModuleList([
#             transforms.ColorJitter(brightness=.3, hue=.2),
#         ]), p=0.3),
#         transforms.RandomGrayscale(p=0.2),
#         transforms.RandomHorizontalFlip(p=0.3),
#         transforms.RandomPerspective(distortion_scale=0.3, p=0.2),
#         transforms.RandomEqualize(p=0.2),
#         transforms.ToTensor(),
#         transforms.Normalize(mean=mean, std=std)
#     ])


# transf_test = transforms.Compose([
#         transforms.Resize(256), 
#         transforms.CenterCrop(224), 
#         transforms.AutoAugment(transforms.AutoAugmentPolicy.SVHN),
#         transforms.RandomInvert(p=0.3),
#         transforms.RandomHorizontalFlip(p=0.2),
#         transforms.RandomGrayscale(p=0.2),
#         transforms.ToTensor(),
#         transforms.Normalize(mean=mean, std=std)
#     ])

In [6]:

# dst_triplet = TripletTrashbin(root=PATH_DST, transform=transf)

# dst_train_triplet, dst_test_triplet = split_into_train_and_test(dst_triplet)

# carico direttamente gli stessi elementi
dst_train_triplet = TripletTrashbin(root='dataset/df_training.csv', transform=transf)
dst_test_triplet = TripletTrashbin(root='dataset/df_test.csv', transform=transf)

triplet_dataset_train_loader = DataLoader(dst_train_triplet, num_workers=NUM_WORKERS, batch_size=BATCH_SIZE, shuffle=True)
triplet_dataset_test_loader = DataLoader(dst_test_triplet, num_workers=NUM_WORKERS, batch_size=BATCH_SIZE)

In [13]:
# TODO: mostra le immagini delle triplette

# 7 minuti e qualcosa ricorda.... ^

#### Primo training:
Alleno la rete con lr=0.002 che è il migliore trovato per SqueezeNet nel precedente progetto.

In [7]:
class TripletNetworkTaskDebugged(pl.LightningModule):
    # lr uguale a quello del progetto vecchio
    def __init__(self, embedding_net, lr=0.002, momentum=0.99, margin=2, num_class=3):
        super(TripletNetworkTaskDebugged, self).__init__()
        self.save_hyperparameters()
        self.embedding_net = embedding_net
        self.criterion = nn.TripletMarginLoss(margin=margin)
        self.num_class = num_class

    def forward(self, x):
        return self.model(x)

    def configure_optimizers(self):
        return SGD(self.embedding_net.parameters(), self.hparams.lr, momentum=self.hparams.momentum)

    # Lightning automatically sets the model to training for training_step and to eval for validation.
    def training_step(self, batch, batch_idx):

        # print("STEP 0: ")

        I_i, I_j, I_k, *_ = batch

        # print(f"i_i: {len(I_i)}, i_j :{len(I_j)}, i_k:{len(I_k)}")

        # print(f"Shape: {I_i.shape}")

        phi_i = self.embedding_net(I_i)
        phi_j = self.embedding_net(I_j)
        phi_k = self.embedding_net(I_k)

        # print(f"phi_i: {phi_i}, phi_j :{phi_j}, phi_k:{phi_k}")

        # calcoliamo la loss
        loss_triplet = self.criterion(phi_i, phi_j, phi_k)
        # print(f"training_step: loss_triplet {loss_triplet}")
        # self.log('train/loss', loss_triplet)
        # return loss_triplet
        
        loss_embedd = phi_i.norm(2) + phi_i.norm(2) + phi_i.norm(2)

        # print(f"loss embedd {loss_embedd}")

        loss = loss_triplet + 0.001 *loss_embedd
        
        # print(f"loss {loss}")

        self.log('train/loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        I_i, I_j, I_k, *_ = batch
        phi_i = self.embedding_net(I_i)
        phi_j = self.embedding_net(I_j)
        phi_k = self.embedding_net(I_k)

        #calcolo la loss
        loss_triplet = self.criterion(phi_i, phi_j, phi_k)
        # print(f"validation_step: loss_triplet {loss_triplet}")
        # self.log('train/loss', loss_triplet)
        # return loss_triplet
        loss_embedd = phi_i.norm(2) + phi_i.norm(2) + phi_i.norm(2)
        loss = loss_triplet + 0.001 * loss_embedd

        self.log('valid/loss', loss)
        return loss


In [8]:
triplet_trashbin_task = TripletNetworkTaskDebugged(squeezeNet_1_0, lr=0.002)
logger = TensorBoardLogger("metric_logs", name="test_trashbin_v1",)

trainer = pl.Trainer(gpus=GPUS, logger = logger, max_epochs = 5, check_val_every_n_epoch = 2, )
trainer.fit(triplet_trashbin_task, triplet_dataset_train_loader, triplet_dataset_test_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name          | Type              | Params
----------------------------------------------------
0 | embedding_net | SqueezeNet        | 735 K 
1 | criterion     | TripletMarginLoss | 0     
----------------------------------------------------
735 K     Trainable params
0         Non-trainable params
735 K     Total params
2.942     Total estimated model params size (MB)


Global seed set to 1996


Epoch 2:   4%|▎         | 3/83 [03:06<1:23:01, 62.27s/it, loss=1.01, v_num=9]   

#### Secondo training

In [23]:
triplet_trashbin_task_v2 = TripletNetworkTask(squeezeNet_1_0, lr=0.002)

checkpoint_callback = [ ModelCheckpoint(
    monitor= 'valid/loss',
    dirpath='/Users/danilo/GitHub/deep-learning/metric_logs/test_trashbin_v1/version_0/',
    filename='epoch=9-step=3299'
) ]

logger = TensorBoardLogger("metric_logs", name="test_trashbin_v1",)
trainer = pl.Trainer(gpus=GPUS, logger = logger, max_epochs = 15, check_val_every_n_epoch = 1, callbacks=checkpoint_callback )
trainer.fit(triplet_trashbin_task_v2, triplet_dataset_train_loader, triplet_dataset_test_loader, ckpt_path='metric_logs/test_trashbin_v1/version_0/checkpoints/epoch=9-step=3299.ckpt')

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
Restoring states from the checkpoint path at metric_logs/test_trashbin_v1/version_0/checkpoints/epoch=9-step=3299.ckpt
/Users/danilo/GitHub/deep-learning/venv/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:248: UserWarning: You're resuming from a checkpoint that ended mid-epoch. Training will start from the beginning of the next epoch. This can cause unreliable results if further training is done, consider using an end of epoch checkpoint.
  "You're resuming from a checkpoint that ended mid-epoch."
Restored all states from the checkpoint file at metric_logs/test_trashbin_v1/version_0/checkpoints/epoch=9-step=3299.ckpt

  | Name          | Type              | Params
----------------------------------------------------
0 | embedding_net | SqueezeNet        | 735 K 
1 | criterion     | TripletMarginLoss | 0     
--------------------------

Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s]

/Users/danilo/GitHub/deep-learning/venv/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:117: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


Global seed set to 1996
/Users/danilo/GitHub/deep-learning/venv/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:117: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


Epoch 10:   0%|          | 0/413 [00:00<?, ?it/s]loss embedd 0.0
loss 2.0
Epoch 10:   0%|          | 1/413 [00:15<1:48:54, 15.86s/it, loss=2, v_num=1]loss embedd 0.0
loss 2.0
Epoch 10:   0%|          | 2/413 [00:28<1:36:11, 14.04s/it, loss=2, v_num=1]loss embedd 0.0
loss 2.0
Epoch 10:   1%|          | 3/413 [00:40<1:31:59, 13.46s/it, loss=2, v_num=1]loss embedd 0.0
loss 2.0
Epoch 10:   1%|          | 4/413 [00:53<1:30:21, 13.26s/it, loss=2, v_num=1]loss embedd 0.0
loss 2.0
Epoch 10:   1%|          | 5/413 [01:05<1:29:29, 13.16s/it, loss=2, v_num=1]loss embedd 0.0
loss 2.0
Epoch 10:   1%|▏         | 6/413 [01:18<1:28:26, 13.04s/it, loss=2, v_num=1]loss embedd 0.0
loss 2.0
Epoch 10:   2%|▏         | 7/413 [01:30<1:27:21, 12.91s/it, loss=2, v_num=1]loss embedd 0.0
loss 2.0
Epoch 10:   2%|▏         | 8/413 [01:42<1:26:34, 12.83s/it, loss=2, v_num=1]loss embedd 0.0
loss 2.0
Epoch 10:   2%|▏         | 9/413 [01:54<1:25:59, 12.77s/it, loss=2, v_num=1]loss embedd 0.0
loss 2.0
Epoch 10:   2%|▏ 

MisconfigurationException: ModelCheckpoint(monitor='val_loss') not found in the returned metrics: ['train/loss', 'valid/loss']. HINT: Did you call self.log('val_loss', value) in the LightningModule?

In [ ]:
#TODO: controlla i commit

In [ ]:
#TODO: devo estrarre TSNE??
# Come continuo ???